# Brazilian E-Commerce Dataset

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Load all the CSVs into DataFrames
orders = pd.read_csv("../data/raw/olist_orders_dataset.csv")
order_reviews = pd.read_csv("../data/raw/olist_order_reviews_dataset.csv")
order_payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")
order_items = pd.read_csv("../data/raw/olist_order_items_dataset.csv")
products = pd.read_csv("../data/raw/olist_products_dataset.csv")
sellers = pd.read_csv("../data/raw/olist_sellers_dataset.csv")
customers = pd.read_csv("../data/raw/olist_customers_dataset.csv")
geolocation = pd.read_csv("../data/raw/olist_geolocation_dataset.csv")
product_category_name_translation = pd.read_csv("../data/raw/product_category_name_translation.csv")

## 2 Extract, Transform, Load

### 2.1 Build Dimension Tables

In [3]:
# EDA showed that geolocation has many duplicate rows
geolocation_unique = geolocation.drop_duplicates()

# Group by (zip_code_prefix, city, state) and take mean coordinates
geolocation_cleaned = (
    geolocation_unique.groupby(["geolocation_zip_code_prefix", "geolocation_city", "geolocation_state"])
    .agg(
        {
            "geolocation_lat": "mean",
            "geolocation_lng": "mean"
        }
    )
    .reset_index()
)

# Rename for clarity
geolocation_cleaned.rename(columns={
    "geolocation_zip_code_prefix": "zip_code_prefix",
    "geolocation_lat": "lat",
    "geolocation_lng": "lng",
    "geolocation_city": "city",
    "geolocation_state": "state"
}, inplace=True)

print(geolocation_cleaned.shape)
geolocation_cleaned.head()

(27912, 5)


,zip_code_prefix,city,state,lat,lng
0,1001,sao paulo,SP,-23.550312,-46.634030
1,1001,são paulo,SP,-23.549998,-46.634060
2,1002,sao paulo,SP,-23.548260,-46.635353
3,1002,são paulo,SP,-23.544641,-46.633180
4,1003,sao paulo,SP,-23.548991,-46.635653


#### 2.1.1 `dim_customers` table

In [4]:
# Drop duplicates based on customer_id
dim_customers = customers.drop_duplicates(subset=["customer_id"]).copy()
# Add surrogate key
dim_customers["customer_key"] = dim_customers.index + 1
# Join with geolocation on to get lat/lng
dim_customers = dim_customers.merge(
    geolocation_cleaned,
    left_on=["customer_zip_code_prefix", "customer_city", "customer_state"],
    right_on=["zip_code_prefix", "city", "state"],
    how="left"
)
# Reorder and drop redundant columns
dim_customers = dim_customers[[
    "customer_key", "customer_id", "customer_unique_id", "customer_zip_code_prefix",
    "customer_city", "customer_state", "lat", "lng"
]]

print(dim_customers.shape)
dim_customers.head()

(99441, 8)


,customer_key,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,lat,lng
0,1,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,-20.499273,-47.396658
1,2,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,-23.728384,-46.542342
2,3,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.531288,-46.656714
3,4,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,-23.500670,-46.186348
4,5,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,-22.975708,-47.143140


#### 2.2.2 `dim_sellers` table

In [5]:
# Drop duplicates based on seller_id
dim_sellers = sellers.drop_duplicates(subset=["seller_id"]).copy()
# Add surrogate key
dim_sellers["seller_key"] = dim_sellers.index + 1
# Join with geolocation on to get lat/lng
dim_sellers = dim_sellers.merge(
    geolocation_cleaned,
    left_on=["seller_zip_code_prefix", "seller_city", "seller_state"],
    right_on=["zip_code_prefix", "city", "state"],
    how="left"
)
# Reorder and drop redundant columns
dim_sellers = dim_sellers[[
    "seller_key", "seller_id", "seller_zip_code_prefix",
    "seller_city", "seller_state", "lat", "lng"
]]

print(dim_sellers.shape)
dim_sellers.head()

(3095, 7)


,seller_key,seller_id,seller_zip_code_prefix,seller_city,seller_state,lat,lng
0,1,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,-22.893317,-47.060596
1,2,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,-22.381828,-46.948224
2,3,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,-22.909446,-43.180240
3,4,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,-23.657111,-46.612556
4,5,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,-22.964857,-46.534080


#### 2.2.3 `dim_products` table

In [6]:
# Dictionary that maps Portuguese category names to English
product_category_name_translation_dict = dict(zip(
    product_category_name_translation["product_category_name"],
    product_category_name_translation["product_category_name_english"]
))

In [7]:
# Drop duplicates based on product_id
dim_products = products.drop_duplicates(subset=["product_id"]).copy()
# Add surrogate key
dim_products["product_key"] = dim_products.index + 1
# Rename misspelled columns
dim_products.rename(columns={
    "product_name_lenght": "product_name_length",
    "product_description_lenght": "product_description_length"
}, inplace=True)
# Translate category names to English
dim_products["product_category_name"] = dim_products["product_category_name"].replace(product_category_name_translation_dict)
# Reorder columns
dim_products = dim_products[[
    "product_key", "product_id", "product_category_name",
    "product_name_length", "product_description_length", "product_photos_qty",
    "product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm"
]]

print(dim_products.shape)
dim_products.head()

(32951, 10)


,product_key,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,2,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,3,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,4,cef67bcfe19066a932b7673e239eb23d,baby,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,5,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0


### 2.3 Builing Fact Tables

#### `fact_order_items` table

In [8]:
# Aggregate installment payments by order_id
payments_agg = order_payments.groupby("order_id").agg({
    "payment_value": "sum",
    "payment_installments": "max"
}).reset_index()
payments_agg.rename(columns={
    "payment_value": "total_payment_value"
}, inplace=True)

print(payments_agg.shape)
payments_agg.head()

(99440, 3)


,order_id,total_payment_value,payment_installments
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,2
1,00018f77f2f0320c557190d7a144bdd3,259.83,3
2,000229ec398224ef6ca0657da4fc703e,216.87,5
3,00024acbcdf0a6daa1e931b038114c75,25.78,2
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04,3


In [9]:
# Convert to review_answer_timestamp to datetime
order_reviews["review_answer_timestamp"] = pd.to_datetime(order_reviews["review_answer_timestamp"])
# Find latest timestamp per order_id
# Returns a Series with the same index and shape as the original
# .transform("max") returns a Series aligned to the original DataFrame index.
latest_ts = order_reviews.groupby("order_id")["review_answer_timestamp"].transform("max")
# Filter to only rows with that timestamp
latest_reviews = order_reviews[order_reviews["review_answer_timestamp"] == latest_ts]
# Compute mean review_score per order_id
latest_review_scores = latest_reviews.groupby("order_id")["review_score"].mean().reset_index()
# Rename it to average_review_score
latest_review_scores.rename(columns={
    "review_score": "average_review_score"
}, inplace=True)

print(latest_review_scores.shape)
latest_review_scores.head()

(98673, 2)


,order_id,average_review_score
0,00010242fe8c5a6d1ba2dd792cb16214,5.0
1,00018f77f2f0320c557190d7a144bdd3,4.0
2,000229ec398224ef6ca0657da4fc703e,5.0
3,00024acbcdf0a6daa1e931b038114c75,4.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,5.0


In [10]:
# Start with order_items
fact_order_items = order_items.copy()
# Join with orders to get customer_id and order datetimes
fact_order_items = fact_order_items.merge(
    orders, on="order_id", how="left"
)
# Add customer surrogate key
fact_order_items = fact_order_items.merge(
    dim_customers[["customer_id", "customer_key"]], on="customer_id", how="left"
)
# Add seller surrogate key
fact_order_items = fact_order_items.merge(
    dim_sellers[["seller_id", "seller_key"]], on="seller_id", how="left"
)
# Add products surrogate key
fact_order_items = fact_order_items.merge(
    dim_products[["product_id", "product_key"]], on="product_id", how="left"
)
# Add payments aggregated info
fact_order_items = fact_order_items.merge(
    payments_agg, on="order_id", how="left"
)
# Add average review score
fact_order_items = fact_order_items.merge(
    latest_review_scores, on="order_id", how="left"
)

print(order_items.shape)
print(orders.shape)
print(fact_order_items.shape)
fact_order_items.head()

(112650, 7)
(99441, 8)
(112650, 20)


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_key,seller_key,product_key,total_payment_value,payment_installments,average_review_score
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,65558,514,25866,72.19,2.0,5.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,34266,472,27231,259.83,3.0,4.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,34956,1825,22625,216.87,5.0,5.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,51764,2024,15404,25.78,2.0,4.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,7603,1598,8863,218.04,3.0,5.0


In [ ]:
# Reorder columns and drop redundant ones